In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans

In [50]:
# Read data
dataset_BJ = pd.read_csv('/Users/liruohan/Documents/DocumentsMacBookPro/2024Spring/StudyPeriod01/DesignOfAISystems/Assignment/module3/Cities/Beijing_labeled.csv')
dataset_SY = pd.read_csv('/Users/liruohan/Documents/DocumentsMacBookPro/2024Spring/StudyPeriod01/DesignOfAISystems/Assignment/module3/Cities/Shenyang_labeled.csv')

dataset_SH = pd.read_csv('/Users/liruohan/Documents/DocumentsMacBookPro/2024Spring/StudyPeriod01/DesignOfAISystems/Assignment/module3/Cities/Shanghai_labeled.csv')
dataset_GZ = pd.read_csv('/Users/liruohan/Documents/DocumentsMacBookPro/2024Spring/StudyPeriod01/DesignOfAISystems/Assignment/module3/Cities/Guangzhou_labeled.csv')


#Train dataset without the PM labels
dataset_BJ_SY = pd.concat([dataset_BJ.iloc[:,:], dataset_SY.iloc[:,:]], ignore_index=True)


In [51]:
#Fit
#Return the relation of cluster indexes and labels
def kmeansFitting(X, Y, k):
    kmeans = KMeans(n_clusters=k, random_state=0) 
    kmeans.fit(X)
    clusters = kmeans.predict(X)
    labels = {}
    for cluster_id in range(k):
        indices = np.where(clusters == cluster_id)[0]
        clusters_labels = Y[indices]        
        label_1_size = clusters_labels.sum()
        label_0_size = clusters_labels.shape[0] - label_1_size
        if(label_0_size > label_1_size):
            labels[cluster_id] = 0
        else:
            labels[cluster_id] = 1
        
    return kmeans,labels

In [52]:
#Predict
def kmeansPredict(X, kmeans, labels):
    
    clusters = kmeans.predict(X)
    Y = np.array([labels[cluster] for cluster in clusters])
        
    return Y

In [53]:
#Score
def evaluate(Y_predicted, Y):
    result = abs(Y_predicted - Y)
    wrong_rate = result.sum()/result.shape[0]
    return 1 - wrong_rate


In [54]:
#Cross validation with c=6
size = dataset_BJ_SY.shape[0]
c = 6
validationSize = int(size/c)
#initialize k
k_0 = 0
# Choose k
accuracy_train = np.zeros(11)
accuracy_validate = np.zeros(11)

for k in range(2,11):
    for i in range(6):
        if (i < 5):
            dataset_validate = dataset_BJ_SY.iloc[i*validationSize : (i+1)*validationSize]
            dataset_train = pd.concat([dataset_BJ_SY.iloc[ : i*validationSize], dataset_BJ_SY.iloc[(i+1)*validationSize : ]], ignore_index=True)
        else:
            dataset_validate = dataset_BJ_SY.iloc[i*validationSize : ]
            dataset_train = pd.concat([dataset_BJ_SY.iloc[ : i*validationSize], dataset_BJ_SY.iloc[(i+1)*validationSize : ]], ignore_index=True)

        #features and labels
        X_train = dataset_train.iloc[:, :-1]
        Y_train = dataset_train.iloc[:, -1]
        X_validate = dataset_validate.iloc[:, :-1]
        Y_validate = dataset_validate.iloc[:, -1]

        #Fit
        kmeans, labels = kmeansFitting(X_train, Y_train, k)
        
        #Validation
        Y_validate_predicted = kmeansPredict(X_validate, kmeans, labels)
        Y_train_predicted = kmeansPredict(X_train, kmeans, labels)
        
        #Score
        acc_validate = evaluate(Y_validate_predicted, Y_validate)
        acc_train = evaluate(Y_train_predicted, Y_train)

        accuracy_validate[k] += acc_validate
        accuracy_train[k] += acc_train

    accuracy_validate[k] = accuracy_validate[k] / c
    accuracy_train[k] = accuracy_train[k] / c

print("Training accuracy: ")
print(accuracy_train[2:])
print("Validation accuracy: ")
print(accuracy_validate[2:])

for k in range(11):
    if accuracy_validate[k] == accuracy_validate.max():
        k_0 = k
        print(k)


Training accuracy: 
[0.72489294 0.72489294 0.72965879 0.729866   0.73532256 0.74519961
 0.75873739 0.75086338 0.74699544]
Validation accuracy: 
[0.72494831 0.72494831 0.73220972 0.7308266  0.7308266  0.71086966
 0.75504413 0.74812422 0.75054469]
8


In [55]:
#Predict with k = k_0 for data in Guangzhou and Shanghai
X_train = dataset_BJ_SY.iloc[:, :-1]
Y_train = dataset_BJ_SY.iloc[:, -1]
kmeans, labels = kmeansFitting(X_train, Y_train, k_0)

X_GZ = dataset_GZ.iloc[:, :-1]
Y_GZ = dataset_GZ.iloc[:, -1]
Y_GZ_predicted = kmeansPredict(X_GZ, kmeans, labels)
accuracy_GZ = evaluate(Y_GZ_predicted,Y_GZ)
print("The predicted value of PM_HIGH for dataset Guangzhou is: " + str(accuracy_GZ))

X_SH = dataset_SH.iloc[:, :-1]
Y_SH = dataset_SH.iloc[:, -1]
Y_SH_predicted = kmeansPredict(X_SH, kmeans, labels)
accuracy_SH = evaluate(Y_SH_predicted,Y_SH)
print("The predicted value of PM_HIGH for dataset Shanghai is: " + str(accuracy_SH))


The predicted value of PM_HIGH for dataset Guangzhou is: 0.9223372781065089
The predicted value of PM_HIGH for dataset Shanghai is: 0.8371576609918578
